# 04. SQLでベクトル検索

### 1. Embeddingモデルを作成
- 現状はSQLだけではベクトルデータが作成できません。
- HuggingFaceのEmbeddingsモデル　ここでは`intfloat/multilingual-e5-large`を使います

- https://huggingface.co/intfloat/multilingual-e5-large


In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from tqdm import tqdm

embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

### 2. Jupyter NotebookでDb2 Magic Commands拡張機能を有効にする
参考: [Jupyter NotebookからDb2に簡単アクセス](https://qiita.com/nishikyon/items/b0d0262950e890349f29)

In [ ]:
import os
# Jupyter NotebookでDb2 Magic Commands拡張機能を有効にする
if not os.path.isfile('db2.ipynb'):
    os.system('wget https://raw.githubusercontent.com/IBM/db2-jupyter/master/db2.ipynb')

%run db2.ipynb

### 3. Db2へ接続

In [ ]:
from dotenv import dotenv_values

# .envファイルを読み込む
# 事前にsample_env を 参考に .envを作成しておく

con_str = dotenv_values(".env")
database = con_str.get("database")
user=con_str.get("uid")
password=con_str.get("pwd")
host=con_str.get("hostname")
port=con_str.get("port")

# SSL接続の場合はSSL TRUEを最後に付与する
%sql CONNECT TO {database} USER {user} USING {password} HOST {host} port {port}

### 6. SQLで類似検索
データができましたので、SQLでベクトル検索してみましょう！

SQLでベクトル検索(==類似検索)するには、[VECTOR_DISTANCE関数](https://www.ibm.com/docs/en/db2/12.1.0?topic=functions-vector-distance)を使って、ベクトル間の距離を出し、距離が近いものほど似ているとして検索します。

```:Syntax
VECTOR_DISTANCE(vector1,vector2, metric)
```

#### 6-1. 人気のイタリア館、人気すぎて入れそうにないので、似たパビリオンないかな？


-> イタリア館のEMBEDDINGとの距離が近いレコードを検索

既に入っているEMBEDDING同士を比較するなら、再度ベクトル化不要!

WHERE文で条件も付加


SQL:
```sql
SELECT 
    ID, カテゴリー, パビリオン名, 主観評価, 事前予約, 見学タイプ, 混雑度, 見どころ, 詳細, おとな向け評価, おとなコメント, 中高生向け評価, 中高生コメント, 子連れ向け評価, 子連れコメント,
    VECTOR_DISTANCE(
        (SELECT EMBEDDING FROM VECTEST.EXPO_INFO WHERE パビリオン名 = 'イタリア  バチカン'),
        EMBEDDING, 
        COSINE
    ) AS DISTANCE
FROM 
    VECTEST.EXPO_INFOでは
WHERE 
    カテゴリー = '海外パビリオン' 
ORDER BY 
    DISTANCE ASC
FETCH FIRST 5 ROWS ONLY
```


In [ ]:
sql_str = f"""
SELECT 
    ID, カテゴリー, パビリオン名, 主観評価, 事前予約, 見学タイプ, 混雑度, 見どころ, 詳細, おとな向け評価, おとなコメント, 中高生向け評価, 中高生コメント, 子連れ向け評価, 子連れコメント,
    VECTOR_DISTANCE(
        (SELECT EMBEDDING FROM VECTEST.EXPO_INFO WHERE パビリオン名 = 'イタリア  バチカン'),
        EMBEDDING, 
        COSINE
    ) AS DISTANCE
FROM 
    VECTEST.EXPO_INFO
WHERE 
    カテゴリー = '海外パビリオン' 
    /* AND 混雑度 NOT LIKE '%混雑%' */
ORDER BY 
    DISTANCE ASC
FETCH FIRST 5 ROWS ONLY
"""

%sql {sql_str}

    
#### 6-2. 「砂漠の砂に座りたい」ので、万博情報から「砂漠の砂に座りたい」で検索


-「砂漠の砂に座りたい」という文字をベクトル化して、そのデータとEXPO_INFOのEMBEDDING列との距離を出して、その距離が近いものが”似ている”ということになります。


- [VECTOR_DISTANCE ](https://www.ibm.com/docs/en/db2/12.1.0?topic=functions-vector-distance)　を使います


基本のSQL:
```
SELECT <取得したい列名1>, <取得したい列名2> ...... 
VECTOR_DISTANCE(<ベクトルデータが入った列名>, 
VECTOR('<ベクトル配列の文字列>',<ベクトル次元数>, <ベクトル値のデータタイプ>), <距離メトリック>) as distance
FROM <テーブル名>
ORDER BY distance
FETCH FIRST <取得したい最大行数> ROWS ONLY;
```


##### 6-2-1. 文字列をベクトル化してクリップボードに貼り付け　かつ　ベクトル値を戻す関数
SQLではベクトル化できないので、以下を使います   

In [ ]:
import pyperclip
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

def get_vector_data(query_str):
    vec_data = embeddings.embed_query(query_str)
    pyperclip.copy(vec_data) # クリップボードに貼り付け
    print("copied!")
    return vec_data  # ベクトルデータを戻す

In [ ]:
# ベクトル化された文字列が、クリップボードにコピーされます
vecdata = get_vector_data("砂漠の砂に座りたい")

DBeaverでやってみましょう!

```sql
SELECT ID, カテゴリー, パビリオン名, 主観評価, 
事前予約, 見学タイプ, 混雑度, 見どころ, 詳細, 
おとな向け評価, おとなコメント, 中高生向け評価, 中高生コメント, 子連れ向け評価, 子連れコメント, 
VECTOR_DISTANCE(EMBEDDING, 
VECTOR('<ベクトル配列の文字列をコピペ>',
1024, FLOAT32), COSINE) as distance
FROM VECTEST.EXPO_INFO
ORDER BY distance
FETCH FIRST 4 ROWS ONLY;
```

In [ ]:
# せっかくなので　pythonでもquery

sql_str = f"""
SELECT ID, カテゴリー, パビリオン名, 主観評価, 
事前予約, 見学タイプ, 混雑度, 見どころ, 詳細, 
おとな向け評価, おとなコメント, 中高生向け評価, 中高生コメント, 子連れ向け評価, 子連れコメント, 
VECTOR_DISTANCE(EMBEDDING, 
VECTOR('{vecdata}',1024, FLOAT32), COSINE) as distance
FROM VECTEST.EXPO_INFO
ORDER BY distance
FETCH FIRST 4 ROWS ONLY;
"""

print(sql_str)

In [ ]:

#sql実行
%sql {sql_str}
